# General Analysis
#### Information on dataset features, distribution, rater agreement, etc.

In [37]:
from typing import Optional, List

import numpy as np
import pandas as pd

import tqdm
import plotly.express as px

import peyes
import analysis.utils as u
from analysis._article_results.hfc._helpers import *

## Load Dataset

In [2]:
dataset = peyes.datasets.hfc(directory=u.DATASETS_DIR, save=False, verbose=True)
dataset.head()

,trial_id,subject_id,stimulus_type,t,x,y,pupil,pixel_size,viewer_distance,subject_group,...,JB,JF,JV,KH,MN,MS,PZ,RA,RH,TC
0,1,4591219856350558064,free_viewing,0.000,966.1,543.6,NaN,0.02652,65,adult,...,1,1,1,0,1,0,1,1,1,1
1,1,4591219856350558064,free_viewing,3.333,967.3,546.8,NaN,0.02652,65,adult,...,1,1,1,1,1,1,1,1,1,1
2,1,4591219856350558064,free_viewing,6.667,967.9,537.5,NaN,0.02652,65,adult,...,1,1,1,1,1,1,1,1,1,1
3,1,4591219856350558064,free_viewing,10.000,969.8,538.8,NaN,0.02652,65,adult,...,1,1,1,1,1,1,1,1,1,1
4,1,4591219856350558064,free_viewing,13.333,971.5,548.6,NaN,0.02652,65,adult,...,1,1,1,1,1,1,1,1,1,1


### Dataset Features
#### (1) Pixel Size, Viewer Distance

In [7]:
viewer_distances = dataset["viewer_distance"].unique()
pixel_sizes = dataset["pixel_size"].unique()

print(f"Viewer Distances (cm):\t{viewer_distances}")
print(f"Pixel Sizes (cm):\t\t{pixel_sizes}")

Viewer Distances (cm):	[65]
Pixel Sizes (cm):		[0.02652028]


In [8]:
VIEWER_DISTANCE_CM = viewer_distances[0]
PIXEL_SIZE_CM = pixel_sizes[0]

PIXEL_SIZE_DEG = peyes._utils.pixel_utils.pixels_to_visual_angle(1, VIEWER_DISTANCE_CM, PIXEL_SIZE_CM)
DEG_IN_PIXEL = peyes._utils.pixel_utils.visual_angle_to_pixels(1, VIEWER_DISTANCE_CM, PIXEL_SIZE_CM)

print(f"Viewer distance:\t\t{10 * VIEWER_DISTANCE_CM}mm")
print(f"Pixel size:\t\t\t\t{10 * PIXEL_SIZE_CM:.3f}mm")
print(f"Pixel size:\t\t\t\t{PIXEL_SIZE_DEG:.3f}° (DVA)")
print(f"1° (DVA) in pixels:\t\t{DEG_IN_PIXEL:.2f}px")

Viewer distance:		650mm
Pixel size:				0.265mm
Pixel size:				0.023° (DVA)
1° (DVA) in pixels:		42.78px


#### (2) Sampling Rates, Types of Stimuli

In [30]:
trials_data = dataset.groupby(peyes.constants.TRIAL_ID_STR)[[peyes.constants.T, peyes.constants.STIMULUS_TYPE_STR]]
sampling_rates = trials_data.apply(
    lambda sub: peyes._utils.event_utils.calculate_sampling_rate(sub[peyes.constants.T].values)
).rename(peyes.constants.SAMPLING_RATE_STR)
stim_types = trials_data.first()[peyes.constants.STIMULUS_TYPE_STR]
trials_data = pd.concat([sampling_rates, stim_types], axis=1).reset_index().groupby(
    [peyes.constants.STIMULUS_TYPE_STR, peyes.constants.SAMPLING_RATE_STR]
).apply(lambda sub: sub.values, include_groups=False).rename(peyes.constants.TRIAL_ID_STR).to_frame().map(
    lambda arr: np.asarray(arr).flatten()
)

trials_data[peyes.constants.COUNT_STR] = trials_data[peyes.constants.TRIAL_ID_STR].apply(lambda arr: len(arr))
trials_data['num_samples'] = trials_data[peyes.constants.TRIAL_ID_STR].apply(
    lambda arr: dataset[np.isin(dataset[peyes.constants.TRIAL_ID_STR], arr)].groupby(peyes.constants.TRIAL_ID_STR).size().sum()
)

trials_data

trial_id  \
stimulus_type sampling_rate                                                      
free_viewing  300.030003                       [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]   
search_task   300.030003     [11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 2...   
              300.030003                  [16, 39, 40, 41, 45, 51, 54, 60, 67]   

                             count  num_samples  
stimulus_type sampling_rate                      
free_viewing  300.030003        10       540216  
search_task   300.030003        51       700728  
              300.030003         9        26748

**To conclude:**  
The sampling rates are equal for all trials (_300Hz_).  
We have 2 types of stimuli:  
- _free viewing_ - recordings of 10 adults freely viewing color images.  
- _search task_ - recordings of infants performing a visual search task.

## Label Counts

In [56]:
GT_LABELERS = [GT1, GT2, GT3, GT4, GT5]


def sample_stats(dataframe: pd.DataFrame, labeler: Optional[str]) -> pd.DataFrame:
    if labeler:
        subset = dataframe[dataframe[labeler].notnull()]
    else:
        subset = dataframe
    counts = pd.concat([
        subset.groupby("stimulus_type").size().rename("num_samples"),
        subset.groupby("stimulus_type")["subject_id"].nunique().rename("num_subjects"),
        subset.groupby("stimulus_type")["trial_id"].nunique().rename("num_trials"),
    ], axis=1)
    total_counts = pd.Series(
        [len(subset), subset["subject_id"].nunique(), subset["trial_id"].nunique()],
        index=counts.columns, name="total"
    )
    counts.loc["total"] = total_counts
    
    if not labeler:
        return counts
    stats = pd.concat([
        subset[labeler].value_counts(dropna=True, normalize=True).sort_index().rename("total"),
        subset.groupby("stimulus_type")[labeler].value_counts(dropna=True, normalize=True).unstack().fillna(0).T
    ], axis=1).T * 100
    stats.index.name = peyes.constants.LABEL_STR
    return pd.concat([counts, stats], axis=1)


def labels_to_events(dataframe: pd.DataFrame):
    trial_ids = dataframe[peyes.constants.TRIAL_ID_STR].unique()
    annotators = set(dataframe.columns).intersection(set(GT_LABELERS))
    event_dict = {}
    for i, trial_id in tqdm.tqdm(enumerate(trial_ids), total=len(trial_ids)):
        trial_data = dataframe[dataframe["trial_id"] == trial_id]
        stim_type, subj_group = trial_data[[peyes.constants.STIMULUS_TYPE_STR, "subject_group"]].values[0]
        t = trial_data[peyes.constants.T].values
        x = trial_data[peyes.constants.X].values
        y = trial_data[peyes.constants.Y].values
        pupil = trial_data[peyes.constants.PUPIL].values
        ps = trial_data[peyes.constants.PIXEL_SIZE_STR].values[0]
        vd = trial_data[peyes.constants.VIEWER_DISTANCE_STR].values[0]
        for annotator in annotators:
            evnts = peyes.create_events(
                labels=trial_data[annotator].values,
                t=t, x=x, y=y, pupil=pupil, pixel_size=ps, viewer_distance=vd,
            )
            evnts = pd.Series(evnts, name=(trial_id, annotator))
            event_dict[(trial_id, stim_type, subj_group, annotator)] = evnts
    event_df = pd.DataFrame(event_dict).T.dropna(axis=0, how='all')
    event_df.index.names = [
        peyes.constants.TRIAL_ID_STR, peyes.constants.STIMULUS_TYPE_STR, peyes.constants.STIMULUS_NAME_STR, "annotator"
    ]
    return event_df


def events_df_to_series(events_df: pd.DataFrame, min_num_samples: int = 2) -> pd.Series:
    events_as_series = events_df.groupby(
        level=np.arange(events_df.index.nlevels).tolist()
    ).apply(
        lambda sub: pd.Series(sub.values.flatten()).dropna()
    )
    events_as_series = events_as_series[events_as_series.map(lambda x: x.num_samples >= min_num_samples)]
    return events_as_series


def count_events(series: pd.Series) -> pd.DataFrame:
    lbl_counts = {}
    for lbl in peyes._DataModels.EventLabelEnum.EventLabelEnum:
        lbl_counts[lbl.name] = _count_events_for_label(series, lbl)
    res = pd.concat(lbl_counts, axis=0, keys=lbl_counts.keys()).dropna(axis=0, how='all')
    res = res.unstack(0).stack(0, future_stack=True)
    
    all_stim_counts = res.groupby(level=1).sum()
    all_stim_counts.index = [("ALL", lblr) for lblr in all_stim_counts.index]
    res = pd.concat([res, all_stim_counts], axis=0)
    # res['total'] = res.sum(axis=1)
    return res


def _count_events_for_label(series: pd.Series, lbl) -> pd.DataFrame:
    sub_series = series[series.map(lambda evnt: evnt.label == lbl)]
    counts = sub_series.groupby(level=[peyes.constants.STIMULUS_TYPE_STR, "annotator"]).size().unstack(1)
    counts["MEAN"] = counts.mean(axis=1)
    return counts

### (1) Sample-Label Counts

In [45]:
label_counts = pd.concat([sample_stats(dataset, gt) for gt in GT_LABELERS], keys=GT_LABELERS, axis=0).reorder_levels([1, 0]).sort_index()
label_counts.index.names = [peyes.constants.STIMULUS_TYPE_STR, peyes.constants.LABELER_STR]

label_mean = label_counts.groupby(peyes.constants.STIMULUS_TYPE_STR).mean()
label_mean.index = pd.MultiIndex.from_product([label_mean.index, ["mean"]], names=[peyes.constants.STIMULUS_TYPE_STR, peyes.constants.LABELER_STR])
label_counts = pd.concat([label_counts, label_mean], axis=0).sort_index()

label_counts

num_samples  num_subjects  num_trials          0  \
stimulus_type labeler                                                     
free_viewing  DN          540216.0          10.0        10.0  12.292861   
              IH          540216.0          10.0        10.0  10.142610   
              JV          540216.0          10.0        10.0  17.122040   
              MN          540216.0          10.0        10.0  15.975832   
              RA          540216.0          10.0        10.0  17.981696   
              mean        540216.0          10.0        10.0  14.703008   
search_task   DN          727476.0          60.0        60.0  34.450621   
              IH          727476.0          60.0        60.0  32.809330   
              JV          727476.0          60.0        60.0  37.738152   
              MN          727476.0          60.0        60.0  36.687396   
              RA          727476.0          60.0        60.0  37.619385   
              mean        727476.0          60.0        60.0  35.860977   
total         DN         1267692.0          70.0        70.0  25.008283   
              IH         1267692.0          70.0        70.0  23.150103   
              JV         1267692.0          70.0        70.0  28.952774   
              MN         1267692.0          70.0        70.0  27.861342   
              RA         1267692.0          70.0        70.0  29.250954   
              mean       1267692.0          70.0        70.0  26.844691   

                               1  
stimulus_type labeler             
free_viewing  DN       87.707139  
              IH       89.857390  
              JV       82.877960  
              MN       84.024168  
              RA       82.018304  
              mean     85.296992  
search_task   DN       65.549379  
              IH       67.190670  
              JV       62.261848  
              MN       63.312604  
              RA       62.380615  
              mean     64.139023  
total         DN       74.991717  
              IH       76.849897  
              JV       71.047226  
              MN       72.138658  
              RA       70.749046  
              mean     73.155309

### (2) Event-Label Counts

In [61]:
events = labels_to_events(dataset)
events_series = events_df_to_series(events, min_num_samples=2)

event_counts = count_events(events_series).sort_index()
event_counts.index.names = [peyes.constants.STIMULUS_TYPE_STR, peyes.constants.LABELER_STR]

event_counts

100%|██████████| 70/70 [00:06<00:00, 10.99it/s]


FIXATION
stimulus_type labeler          
ALL           DN         8710.0
              IH         9113.0
              JV         9180.0
              MEAN       8862.6
              MN         8648.0
              RA         8662.0
free_viewing  DN         4878.0
              IH         5154.0
              JV         5021.0
              MEAN       4969.0
              MN         4877.0
              RA         4915.0
search_task   DN         3832.0
              IH         3959.0
              JV         4159.0
              MEAN       3893.6
              MN         3771.0
              RA         3747.0